# 정수 인코딩(Integer Encoding)
- 컴퓨터는 텍스트보다는 숫자를 더 잘 처리
- 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업

<br>
<b>1) dictionary 사용하기</b><br>
 (1) 문단을 read<br>
 (2) sent_tokenize를 통해 문장 토큰화<br>
 (3) stop word 정의<br>
 (4) 문장 토큰화를 for문을 통해 단어 토큰화<br>
 (5) 단어 토큰화 전처리 진행<br>
   - 소문자로 변경하여 단어 중복 줄임<br>
   - 짧은 단어 제거<br>
   - stop word를 통해 불용어 제거<br>
 (5-1) 문장별로 단어 리스트 만들어짐<br>
 (6) 이때 단어별로 빈도수 카운팅 why? 소팅하여 숫자화(인덱스)하려고<br>
 (7) 빈도수 높은 순서대로 정렬<br>
 (8) 높은 빈도수를 가진 단어일수록 낮은 인덱스 부여<br>
 (9) 5-1)에 정보를 통해 각 단어들 인덱스 부여<br>
 (9-1) 그런데 인덱스가 없는 것은 인덱스 + 1로 저장시킴<br>
 (9-2) try, except를 이용하여 인덱스가 있으면 인덱스번호를 리스트에 저장하고 인덱스가 없으면 인덱스 len에 + 1로 리스트에 저장<br>
 (9-3) 저장된 리스트를 모두 하나의 리스트로 넣어줌<br>

In [22]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
text="A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [23]:
# 문장 토큰화
text=sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [24]:
# 정제와 단어 토큰화
vocab={} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence = word_tokenize(i)  # 단어 토근화
    result = []
    
    for word in sentence:
        word = word.lower() #소문자화 : why 단어개수 줄이기 위해
        if word not in stop_words: # 불용어 제거
            if len(word) > 2:   #짧은 단어 제거
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1
    sentences.append(result)
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록

In [25]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [26]:
print(vocab["barber"])

8


In [27]:
# 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [28]:
# 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스 부여
word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted:
    if frequency > 1:     # 정제(cleaning) 빈도수가 적은 단어 제외
        i = i + 1
        word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [34]:
vocab_size = 5
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1]
'''
words_frequency = []
for w, c in word_to_index.items():
    if c >= vocab_size + 1:
        words_frequency.append(w)
'''
for w in words_frequency:
    del word_to_index[w]   # 해당 단어에 대한 인덱스 정보를 삭제
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- word_to_index를 사용하여 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾸는 작업
예)<br>
1) sentences에서 첫번째 문장은 ['barber', 'person'] -> [1, 5]로 인코딩<br>
2) 그런데 두번째 문장인 ['barber', 'good', 'person']에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어 존재<br>
3) index가 되어 있지 않은 단어들 OOV(Out of Vocabulary)라고 함.<br>
따라서, word_to_index에 'OOV'란 단어를 추가하고 인덱스로 인코딩하야 함.

In [35]:
word_to_index['OOV'] = len(word_to_index) + 1

In [36]:
encoded=[]
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


<b>2) Counter 사용하기</b><br>

In [59]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
text="A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

# 문장 토큰화
text=sent_tokenize(text)
#print(text)

# 정제와 단어 토큰화
vocab={} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
    sentence = word_tokenize(i)  # 단어 토근화
    result = []
    
    for word in sentence:
        word = word.lower() #소문자화 : why 단어개수 줄이기 위해
        if word not in stop_words: # 불용어 제거
            if len(word) > 2:   #짧은 단어 제거
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1
    sentences.append(result)
#print(sentences)

In [60]:
from collections import Counter
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [61]:
words = sum(sentences, [])
#same code : words = np.hstack(sentences)
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


- Counter()의 입력으로 사용하면 중복을 제거하고 단어의 빈도수를 기록

In [62]:
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [63]:
print(vocab["barber"])

8


- most_common()는 상위 빈도수를 가진 주어진 수의 단어만을 리턴

In [64]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


- 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여

In [65]:
word_to_index = {}
i = 0
for (word, frequency) in vocab:
    i = i + 1
    word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [66]:
word_to_index['OOV'] = len(word_to_index) + 1

In [67]:
encoded=[]
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


<b>3) NLTK의 FreqDist 사용하기</b><br>
- NLTK에서는 빈도수 계산 도구인 FreqDist()지원

In [75]:
from nltk import FreqDist
import numpy as np

In [76]:
#print(sentences)
vocab = FreqDist(np.hstack(sentences))   #np.hstack은 문장 구분을 제거
print(vocab["barber"])

8


- most_common()을 통해 상위 빈도수를 가진 주어진 수의 단어만 리턴

In [77]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [78]:
word_to_index = {}
i = 0
for (word, frequency) in vocab:
    i = i + 1
    word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [79]:
word_to_index['OOV'] = len(word_to_index) + 1
encoded=[]
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


<b>4) enumerate 이해하기</b><br>
- 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴

In [80]:
test = ['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test):
    print("value : {}, index :{}".format(value, index))

value : a, index :0
value : b, index :1
value : c, index :2
value : d, index :3
value : e, index :4
